# **SIN 393 – Introduction to Computer Vision (2023)**

# Lecture 04 - Part 4 - Artificial Neural Networks

Prof. João Fernando Mari ([*joaofmari.github.io*](https://joaofmari.github.io/))

---

## Importing the required libraries
---

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.axes_grid1 import make_axes_locatable

import ipywidgets as widgets

from sklearn import datasets, metrics, preprocessing, model_selection

### %matplotlib widget

In [2]:
def error_brute_force(X, y, w0=[-1, 1], w1=[-1, 1], b=[-1, 1], step=0.1):
    """
    20 x 20 x 20 = 8,000 parameter combinations
    """
    # Initialize error space
    error_space = np.zeros([len(np.arange(w0[0], w0[1], step)),
                            len(np.arange(w1[0], w1[1], step)),
                            len(np.arange(b[0], b[1], step))])

    # Iterate along w0
    for i, w0_ in enumerate(np.arange(w0[0], w0[1], step)):
        # Iterate along w1
        for j, w1_ in enumerate(np.arange(w1[0], w1[1], step)):
            # Iterate along b
            for k, b_ in enumerate(np.arange(b[0], b[1], step)):
                # Initialize epoch error
                erro_epoca = 0.
                w = [w0_, w1_]
                # Iterate along dataset (one epoch)
                for x_, y_ in zip(X, y):
                    # Innet product
                    v = np.dot(x_, w) + b_
                    # Activation function
                    y_out = np.where(v >= 0., 1, 0)

                    # Error
                    erro = y_ - y_out
                    # Update epoch error
                    erro_epoca = erro_epoca + erro**2
            
                # Erro total da época.
                erro_epoca = erro_epoca / 2.
                error_space[i, j, k] = erro_epoca      

    return error_space 

## Dataset
---

In [3]:
# Data
X_bin = np.array([[0, 0],
                  [0, 1],
                  [1, 0],
                  [1, 1]])

# Binary functions
# ================
# AND
y_and = np.array([0, 0, 0, 1])

# OR
y_or = np.array([0, 1, 1, 1])

# XOR
y_xor = np.array([0, 1, 1, 0])

In [4]:
# Loading the IRIS dataset
# ========================
iris = datasets.load_iris()

# Selects only the Setosa (0) and Virginica (1) classes.
# Select only 2 attributes: sepal length (0) and sepal width (1)
X_iris = iris.data[iris.target < 2, :2]
y_iris = iris.target[iris.target < 2]

## Select the data

In [5]:
X = X_bin
y = y_and
str_title = 'AND'

## Model

In [6]:
error_space = error_brute_force(X, y)

print(error_space.shape)

(20, 20, 20)


In [7]:
def plot_error_a(error_space, w0_, w1_, b_, w0=[-1, 1], w1=[-1, 1], b=[-1, 1], step=0.1, surf=True):

    # Convert index to value.
    w0_r = np.arange(w0[0], w0[1], step)
    w1_r = np.arange(w1[0], w1[1], step)
    b_r = np.arange(b[0], b[1], step)

    w0_v = w0_r[w0_]
    w1_v = w1_r[w1_]
    b_v = b_r[b_]

    # Avoid zero division
    if w1_v == w0_v or w1_v == b_v:
        w1_v = w1_v + 0.01

    w = [w0_v, w1_v]

    # Define os pontos extremos do segmento da superficie de decisão plotado.
    # p0 = (x0_min, f_x0_min); p1 = (x0_max, f_x0_max)
    # ----------
    x0_min = X.min() - 1.
    x0_max = X.max() + 1.
    # ----------
    f_x0_min = -(w[0] / w[1]) * x0_min - (b_v / w[1])
    f_x0_max = -(w[0] / w[1]) * x0_max - (b_v / w[1])

    colors = ['r', 'g', 'b', 'y', 'c', 'm']

    print(f'w0: {w0_v:.2f} w1: {w1_v:.2f} b: {b_v:.2f} Error: {error_space[w0_,w1_,b_]}')

    # Plot
    fig = plt.figure(figsize=(12, 3))

    # Feature space
    ax1 = fig.add_subplot(1, 4, 1)
    for y_ in np.unique(y):
        ax1.scatter(X[y==y_][:,0], X[y==y_][:,1], color=colors[y_], label=str(y_))
 
    ax1.set_xlabel('$x_0$')
    ax1.set_ylabel('$x_1$')
    ax1.legend()
    ax1.set_title(str_title)

    ax1.set_xlim(X.min()-.5, X.max()+.5)
    ax1.set_ylim(X.min()-.5, X.max()+.5)

    # Plot decision surface
    ax1.plot([x0_min, x0_max], [f_x0_min, f_x0_max], color='b')
    ax1.set_aspect('equal')

    # w0 Vs. w1
    if surf:
        ax2 = fig.add_subplot(1, 4, 2, projection='3d')
        xx, yy = np.mgrid[0:error_space[:,:,b_].shape[0], 0:error_space[:,:,b_].shape[1]]
        ax2.plot_surface(xx, yy, error_space[:,:,b_] ,rstride=1, cstride=1, linewidth=0)
        ax2.plot([w0_, w0_], [w1_, w1_], [0, np.max(error_space[:,:,b_]) + 1.0], 'r')
    else:
        ax2 = fig.add_subplot(1, 4, 2)
        im2 = ax2.imshow(error_space[:,:,b_], cmap='gray')
        ax2.plot(w0_, w1_, marker='o', color='r')
        # Colorbar
        divider = make_axes_locatable(ax2)
        cax = divider.append_axes('right', size='5%', pad=0.05)
        fig.colorbar(im2, cax=cax, orientation='vertical')
    ax2.set_title('$w_0$ Vs. $w_1$')
    ax2.set_xlabel('$w_0$')
    ax2.set_ylabel('$w_1$')

    # w0 Vs. b
    if surf:
        ax3 = fig.add_subplot(1, 4, 3, projection='3d')
        # create the x and y coordinate arrays (here we just use pixel indices)
        xx, yy = np.mgrid[0:error_space[:,w1_,:].shape[0], 0:error_space[:,:,b_].shape[1]]
        ax3.plot_surface(xx, yy, error_space[:,w1_,:] ,rstride=1, cstride=1, linewidth=0)
        ax3.plot([w0_, w0_], [b_, b_], [0, np.max(error_space[:,w1_,:]) + 1.0], 'r')
    else:
        ax3 = fig.add_subplot(1, 4, 3)
        im3 = ax3.imshow(error_space[:,w1_,:].T, cmap='gray')
        ax3.plot(w0_, b_,  marker='o', color='r')
        # Colorbar
        divider = make_axes_locatable(ax3)
        cax = divider.append_axes('right', size='5%', pad=0.05)
        fig.colorbar(im3, cax=cax, orientation='vertical')
    ax3.set_title('$w_0$ Vs. $b$')
    ax3.set_xlabel('$w_0$')
    ax3.set_ylabel('$b$')
    
    # w1 Vs. b
    if surf:
        ax4 = fig.add_subplot(1, 4, 4, projection='3d')
        # create the x and y coordinate arrays (here we just use pixel indices)
        xx, yy = np.mgrid[0:error_space[w0_,:,:].shape[0], 0:error_space[:,:,b_].shape[1]]
        ax4.plot_surface(xx, yy, error_space[w0_,:,:] ,rstride=1, cstride=1, linewidth=0)
        ax4.plot([w1_, w1_], [b_, b_], [0, np.max(error_space[w0_,:,:]) + 1.0], 'r')
    else:
        ax4 = fig.add_subplot(1, 4, 4)
        im4 = ax4.imshow(error_space[w0_,:,:].T, cmap='gray')
        ax4.plot(w1_, b_,  marker='o', color='r')
        # Colorbar
        divider = make_axes_locatable(ax4)
        cax = divider.append_axes('right', size='5%', pad=0.05)
        fig.colorbar(im4, cax=cax, orientation='vertical')
    ax4.set_title('$w_1$ Vs. $b$')
    ax4.set_xlabel('$w_1$')
    ax4.set_ylabel('$b$')

    plt.tight_layout()

In [8]:
def plot_error_b(error_space, w0_, w1_, b_, w0=[-1, 1], w1=[-1, 1], b=[-1, 1], step=0.1, surf=True):

    # Convert index to value.
    w0_r = np.arange(w0[0], w0[1], step)
    w1_r = np.arange(w1[0], w1[1], step)
    b_r = np.arange(b[0], b[1], step)

    w0_v = w0_r[w0_]
    w1_v = w1_r[w1_]
    b_v = b_r[b_]

    # Avoid zero division
    if w1_v == w0_v or w1_v == b_v:
        w1_v = w1_v + 0.01

    w = [w0_v, w1_v]

    # Define os pontos extremos do segmento da superficie de decisão plotado.
    # p0 = (x0_min, f_x0_min); p1 = (x0_max, f_x0_max)
    # ----------
    x0_min = X.min() - 1.
    x0_max = X.max() + 1.
    # ----------
    f_x0_min = -(w[0] / w[1]) * x0_min - (b_v / w[1])
    f_x0_max = -(w[0] / w[1]) * x0_max - (b_v / w[1])

    colors = ['r', 'g', 'b', 'y', 'c', 'm']

    print(f'w0: {w0_v:.2f} w1: {w1_v:.2f} b: {b_v:.2f} Error: {error_space[w0_,w1_,b_]}')

    fig = plt.figure(figsize=(12, 6))

    # Feature space
    ax1 = fig.add_subplot(2, 3, 1)
    for y_ in np.unique(y):
        ax1.scatter(X[y==y_][:,0], X[y==y_][:,1], color=colors[y_], label=str(y_))
 
    ax1.set_xlabel('$x_0$')
    ax1.set_ylabel('$x_1$')
    ax1.legend()
    ax1.set_title(str_title)

    ax1.set_xlim(X.min()-.5, X.max()+.5)
    ax1.set_ylim(X.min()-.5, X.max()+.5)

    # Plot decision surface
    ax1.plot([x0_min, x0_max], [f_x0_min, f_x0_max], color='b')
    ax1.set_aspect('equal')

    ax1a = fig.add_subplot(2, 3, 2)
    ax1a.axis('off')
    ax1b = fig.add_subplot(2, 3, 3)
    ax1b.axis('off')

    # w0 Vs. w1
    if surf:
        ax2 = fig.add_subplot(2, 3, 4, projection='3d')
        # create the x and y coordinate arrays (here we just use pixel indices)
        xx, yy = np.mgrid[0:error_space[:,:,b_].shape[0], 0:error_space[:,:,b_].shape[1]]
        ax2.plot_surface(xx, yy, error_space[:,:,b_] ,rstride=1, cstride=1, linewidth=0)
        ax2.plot([w0_, w0_], [w1_, w1_], [0, np.max(error_space[:,:,b_]) + 1.0], 'r')
    else:
        ax2 = fig.add_subplot(2, 3, 4)
        im2 = ax2.imshow(error_space[:,:,b_], cmap='gray')
        ax2.plot(w0_, w1_,  marker='o', color='r')
        # Colorbar
        divider = make_axes_locatable(ax2)
        cax = divider.append_axes('right', size='5%', pad=0.05)
        fig.colorbar(im2, cax=cax, orientation='vertical')
    ax2.set_title('$w_0$ Vs. $w_1$')
    ax2.set_xlabel('$w_0$')
    ax2.set_ylabel('$w_1$')

    # w0 Vs. b
    if surf:
        ax3 = fig.add_subplot(2, 3, 5, projection='3d')
        # create the x and y coordinate arrays (here we just use pixel indices)
        xx, yy = np.mgrid[0:error_space[:,w1_,:].shape[0], 0:error_space[:,:,b_].shape[1]]
        ax3.plot_surface(xx, yy, error_space[:,w1_,:] ,rstride=1, cstride=1, linewidth=0)
        ax3.plot([w0_, w0_], [b_, b_], [0, np.max(error_space[:,w1_,:]) + 1.0], 'r')
    else:
        ax3 = fig.add_subplot(2, 3, 5)
        im3 = ax3.imshow(error_space[:,w1_,:].T, cmap='gray')
        ax3.plot(w0_, b_,  marker='o', color='r')
        # Colorbar
        divider = make_axes_locatable(ax3)
        cax = divider.append_axes('right', size='5%', pad=0.05)
        fig.colorbar(im3, cax=cax, orientation='vertical')
    ax3.set_title('$w_0$ Vs. $b$')
    ax3.set_xlabel('$w_0$')
    ax3.set_ylabel('$b$')

    # w1 Vs. b
    if surf:
        ax4 = fig.add_subplot(2, 3, 6, projection='3d')
        xx, yy = np.mgrid[0:error_space[w0_,:,:].shape[0], 0:error_space[:,:,b_].shape[1]]
        ax4.plot_surface(xx, yy, error_space[w0_,:,:] ,rstride=1, cstride=1, linewidth=0)
        ax4.plot([w1_, w1_], [b_, b_], [0, np.max(error_space[w0_,:,:]) + 1.0], 'r')
    else:
        ax4 = fig.add_subplot(2, 3, 6)
        im4 = ax4.imshow(error_space[w0_,:,:].T, cmap='gray')
        ax4.plot(w1_, b_,  marker='o', color='r')
        # Colorbar
        divider = make_axes_locatable(ax4)
        cax = divider.append_axes('right', size='5%', pad=0.05)
        fig.colorbar(im4, cax=cax, orientation='vertical')
    ax4.set_title('$w_1$ Vs. $b$')
    ax4.set_xlabel('$w_1$')
    ax4.set_ylabel('$b$')
    
    plt.tight_layout()

In [9]:
slider_w0 = widgets.IntSlider(value=10, min=0, max=20)
slider_w1 = widgets.IntSlider(value=10, min=0, max=20)
slider_b = widgets.IntSlider(value=10, min=0, max=20)

widgets.interact(plot_error_a, error_space=widgets.fixed(error_space), 
                 w0_=slider_w0, w1_=slider_w1, b_=slider_b, 
                 w0=widgets.fixed([-1, 1]), w1=widgets.fixed([-1, 1]), b=widgets.fixed([-1, 1]), 
                 step=widgets.fixed(0.1), surf=widgets.fixed(False))

interactive(children=(IntSlider(value=10, description='w0_', max=20), IntSlider(value=10, description='w1_', m…

<function __main__.plot_error_a(error_space, w0_, w1_, b_, w0=[-1, 1], w1=[-1, 1], b=[-1, 1], step=0.1, surf=True)>

## Bibliography
---

* GONZALEZ, R.C.; WOODS, R.E. Digital Image Processing. 3rd ed. Pearson, 2007.